## Model buildiing
- Notebook used to validate feature selection for logistic regression 
- Find features
- Build model
- Check accuracy score
- Output pickle model 
- output empty datrame with dummy columns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import numpy as np

%matplotlib inline


#### Import dataset

In [ ]:
df = pd.read_csv('../backend/mapping_heat/fixtures/pitching_data.csv')

In [ ]:
df.columns

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.shape

### Find features to use for model

#### Pitchers

In [ ]:
pitcheres = df.player_name.value_counts().to_frame().reset_index().rename(columns={'index': 'pitcher', 'player_name': 'count'})
pitcheres.head()

In [7]:
pitcheres['count'].mean()

359.5961800818554

In [8]:
pitcheres['count'].min()

2

In [9]:
pitcheres['count'].describe()

count     733.000000
mean      359.596180
std       299.813281
min         2.000000
25%       118.000000
50%       302.000000
75%       470.000000
max      1408.000000
Name: count, dtype: float64

In [10]:
df.player_name.value_counts().to_frame().mean()

player_name    359.59618
dtype: float64

#### Release speed

In [11]:
df.release_speed.value_counts().head(10)

93.5    2154
93.9    2142
93.1    2142
93.3    2128
93.4    2123
93.8    2120
93.2    2113
94.1    2109
92.7    2100
92.9    2096
Name: release_speed, dtype: int64

In [12]:
rl_speed = df.release_speed.value_counts().to_frame().reset_index().rename(columns={'index': 'speed', 'release_speed': 'count'})

In [13]:
rl_speed.speed.describe()

count    466.000000
mean      77.944635
std       14.872838
min       38.100000
25%       67.125000
50%       78.750000
75%       90.375000
max      102.200000
Name: speed, dtype: float64

In [14]:
rl_speed.loc[rl_speed.speed.isnull()]

,speed,count


#### Pitch outcome

In [15]:
df.type.value_counts()

S    121964
B     97648
X     43972
Name: type, dtype: int64

#### Outcome per at bat

In [16]:
df.events.value_counts()

field_out                     25530
strikeout                     15543
single                         9071
walk                           5890
double                         2823
home_run                       2304
force_out                      1308
grounded_into_double_play      1235
hit_by_pitch                    821
field_error                     513
sac_fly                         399
triple                          241
double_play                     150
fielders_choice                 149
sac_bunt                        126
fielders_choice_out             118
caught_stealing_2b               71
strikeout_double_play            43
interf_def                       35
pickoff_caught_stealing_2b       14
pickoff_1b                       12
other_out                        11
pickoff_2b                        7
caught_stealing_3b                6
caught_stealing_home              4
sac_fly_double_play               3
triple_play                       2
pickoff_caught_stealing_3b  

#### Description per pitch outcome

In [17]:
df.description.value_counts()

ball                       89937
foul                       44716
called_strike              44320
hit_into_play              28264
swinging_strike            28026
hit_into_play_no_out        9685
blocked_ball                6874
hit_into_play_score         6029
foul_tip                    2427
swinging_strike_blocked     1968
hit_by_pitch                 821
foul_bunt                    434
missed_bunt                   62
pitchout                      16
bunt_foul_tip                  5
Name: description, dtype: int64

###### Pitch zones

In [18]:
df.zone

0          1.0
1         13.0
2         13.0
3         13.0
4         13.0
          ... 
263579     5.0
263580     7.0
263581     9.0
263582    11.0
263583    11.0
Name: zone, Length: 263584, dtype: float64

In [19]:
df[df.zone.isnull()][['description']]

,description
20677,ball
20682,ball
70022,hit_into_play
70023,blocked_ball
70024,called_strike
...,...
233601,foul
233738,foul
234935,ball
234937,ball


In [20]:
df.pitch_type.value_counts()

FF    89618
SL    46849
SI    42768
CH    31131
CU    24679
FC    17404
KC     5939
FS     4528
CS      191
KN       23
FO        3
Name: pitch_type, dtype: int64

##### Pitch legend

- AB Automatic Ball
- AS Automatic Strike
- CH Change-​up
- CU Curveball
- EP Eephus
- FC Cutter
- FF Four-Seam Fastball
- FO Forkball
- FS Splitter
- FT Two-Seam Fastball (synonymous with SI)
- GY Gyroball
- IN Intentional Ball
- KC Knuckle Curve
- KN Knuckleball
- NP No Pitch
- PO Pitchout
- SC Screwball
- SI Sinker (synonymous with FT)
- SL Slider
- UN Unknown

In [21]:
df.groupby(['pitch_type', 'pitch_name']).size().to_frame().reset_index().rename(columns={0: 'count'}).sort_values(by='count', ascending=False)

,pitch_type,pitch_name,count
4,FF,4-Seam Fastball,89618
10,SL,Slider,46849
9,SI,Sinker,42768
0,CH,Changeup,31131
2,CU,Curveball,24679
3,FC,Cutter,17404
7,KC,Knuckle Curve,5939
6,FS,Split-Finger,4528
1,CS,Curveball,191
8,KN,Knuckleball,23


In [22]:
# x = df[~df.zone.isnull() & ~df.release_spin_rate.isnull()].copy()
x = df[~df.zone.isnull()].copy()

In [23]:
x.shape

(263393, 90)

##### Hot encode non numeric values for y values

In [24]:
def hot_encode_des(x):
    if x == 'hit_into_play_no_out' or x == 'hit_into_play_score':
        return 1
    return 0

In [25]:
def hot_encode_event(x):
    if x == 'single' or x == 'double' or x == 'triple' or x == 'home_run':
        return 1
    return 0

In [26]:
x['norm_zone'] = x['zone'].astype(str)

### Selected features

In [27]:
x_norm = x[['player_name', 'pitch_type', 'release_speed', 'norm_zone']]
# x_norm = x[['release_spin_rate', 'pitch_type', 'release_speed', 'norm_zone']]
# x_norm = x[[ 'pitch_type',  'norm_zone']]


In [28]:
# x_norm[x_norm.release_speed.isnull()]

In [29]:
y = x['description'].apply(lambda x: hot_encode_des(x))

##### Hot encode non numeric data

In [30]:
x = pd.get_dummies(x_norm)

##### Look at y values

In [31]:
hits = [i for i in y if i == 1]
print(len(hits))
print(len(y))
print((len(hits)/len(y) )* 100) 

15706
263393
5.962952698059554


##### Split data

In [32]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

##### view train data

In [33]:
X_train.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

##### View shape

In [34]:
X_train.shape

(184375, 758)

In [35]:
X_test.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

#### Build probability model

In [36]:
clf = LogisticRegression(max_iter=1000, solver='saga', penalty='elasticnet', l1_ratio=0.5)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
y_pred = pd.Series(clf.predict(X_test), index=y_test.index)

#### As expected, the classification is predicting all zeros (no hit).
- We don't really care about the classification since we want to find the probilities
- We'll use the classification to score the overall model
- We can then testt lowering P value for hit and see how it changes accuracy

In [ ]:
pred

In [ ]:
for i in pred:
    if i == 1:
        print("hit")

##### Check accuracy score of model 

In [ ]:
score = metrics.accuracy_score(y_test, pred)

In [ ]:
score

In [ ]:
metrics.precision_score(y_pred, y_test, average='micro')

In [ ]:
metrics.f1_score(y_pred, y_test, average='weighted')

In [ ]:
# cross_val_score(clf, x, y, cv=5) 

###### describe the relationship between a predictor variable and the response. 

In [ ]:
clf.coef_[0]

In [ ]:
from sklearn.metrics import precision_recall_curve

pred_y=clf.predict(X_test) 
probs_y=clf.predict_proba(X_test) 
precision, recall, thresholds = precision_recall_curve(y_test, probs_y[:, 1])
pr_auc = metrics.auc(recall, precision)

plt.figure(figsize=(10,5))
plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="best")
plt.ylim([0,1])

In [ ]:
pred_proba_df = pd.DataFrame(clf.predict_proba(X_test))
accuracies = []
accuracy2 = {}
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
    test_accuracy = metrics.accuracy_score(y_test.to_numpy().reshape(y_test.to_numpy().size,1),
                                           y_test_pred.iloc[:,1].to_numpy().reshape(y_test_pred.iloc[:,1].to_numpy().size,1))
    print('Our testing accuracy is {}'.format(test_accuracy))
    accuracies.append(test_accuracy)
    accuracy2[test_accuracy]  = i
    
    print(metrics.confusion_matrix(y_test.to_numpy().reshape(y_test.to_numpy().size,1),
                           y_test_pred.iloc[:,1].to_numpy().reshape(y_test_pred.iloc[:,1].to_numpy().size,1)))
    

scores = pd.Series(accuracies, index=range(0,20), name="Score") 
scores2 = pd.DataFrame([accuracy2])


In [ ]:
scores
s = scores2.T.reset_index().rename(columns={'index': 'score', 0: 'value'})

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(0, 20), x=s.score, y=s.value, marker='o', markerfacecolor='red', markersize=5)
plt.xlabel('prob of hit')
plt.ylabel('Accuracy')
plt.title('Model response to lowering probability split');

###### Find the probs 

In [ ]:
probs = clf.predict_proba(X=X_test)

In [ ]:
probs = probs * 100

In [ ]:
hit_prob = pd.DataFrame(probs, columns=['no-hit', 'hit'])

In [ ]:
hit_prob.head()

In [ ]:
hit_prob.shape

###### Build out dataframe with probs and outcomes

In [ ]:
x_df = X_test.copy()

In [ ]:
y_df = pd.DataFrame(y_test)

In [ ]:
y_df

In [ ]:
x_df['description'] = y_df

In [ ]:
x_df.head()[['release_speed', 'description' ]]

In [ ]:
x_df.reset_index(inplace=True)

In [ ]:
final_df = x_df.join(hit_prob)

#### Find the higest probability of a hit

In [ ]:
final_df.hit.max()

In [ ]:
final_df[final_df.hit == final_df.hit.max()][['index', 'release_speed', 'description', 'no-hit', 'hit']]

In [ ]:
df.iloc[[20673]][['pitch_type', 'pitch_name','release_speed', 'zone', 'events', 'description']]

#### find prob of all X

In [39]:
all_x = df.copy()

In [40]:
all_x['norm_zone'] = all_x['zone'].astype(str)

In [41]:
features = all_x[['player_name', 'pitch_type', 'release_speed', 'norm_zone']]

In [42]:
features[features.player_name.isnull() == True]

,player_name,pitch_type,release_speed,norm_zone


In [43]:
features[features.release_speed.isnull() == True]['player_name'].value_counts()

Gerrit Cole          114
Thomas Eshelman       41
Alex Cobb             28
Taijuan Walker         2
Brock Holt             2
Yoan Lopez             1
Richard Rodriguez      1
Evan Marshall          1
Kris Bubic             1
Name: player_name, dtype: int64

In [44]:
features[features.pitch_type.isnull() == True]

,player_name,pitch_type,release_speed,norm_zone
20677,Brock Holt,NaN,NaN,nan
20682,Brock Holt,NaN,NaN,nan
70022,Gerrit Cole,NaN,NaN,nan
70023,Gerrit Cole,NaN,NaN,nan
70024,Gerrit Cole,NaN,NaN,nan
...,...,...,...,...
233601,Evan Marshall,NaN,NaN,nan
233738,Kris Bubic,NaN,NaN,nan
234935,Taijuan Walker,NaN,NaN,nan
234937,Taijuan Walker,NaN,NaN,nan


##### Train the entire model

In [37]:
clf.fit(x, y)

/Users/brandonl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.5, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='elasticnet',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
features.dropna(inplace=True)

/Users/brandonl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
encoded_x = pd.get_dummies(features)

In [47]:
probs = clf.predict_proba(encoded_x)

In [48]:
probs = probs*100

In [49]:
hit_prob = pd.DataFrame(probs, columns=['no-hit', 'hit'])

In [50]:
hit_prob

,no-hit,hit
0,95.724723,4.275277
1,98.429854,1.570146
2,98.430485,1.569515
3,98.430012,1.569988
4,98.431432,1.568568
...,...,...
263128,86.839934,13.160066
263129,91.606337,8.393663
263130,91.762448,8.237552
263131,98.253755,1.746245


In [51]:
hit_prob.max()

no-hit    99.394054
hit       28.607610
dtype: float64

In [52]:
encoded_x.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

In [53]:
encoded_x.shape

(263133, 758)

In [54]:
output_x = encoded_x.copy()
out = output_x.iloc[0:0] 

In [55]:
out.to_csv('./df_shell.csv')

In [56]:
# save the model to disk
filename = 'pitch_prob.sav'
pickle.dump(clf, open(filename, 'wb'))